In [1]:
import pandas as pd
import numpy as np
from model_settings import ms
from historical_av_key_collector import keys_df, symbol
from historical_av_underlying_fetcher import historical_spots

keys_df = keys_df[keys_df['surface_key'].isna()].dropna(subset=['raw_data_key']).iloc[:1]

contracts = {}


for i,row in keys_df.iterrows():
    with pd.HDFStore(f"alphaVantage {symbol}.h5") as store:
        key = row['raw_data_key']
        date = key[key.find('_',0)+1:key.find("/",1)].replace("_","-")
        contracts[date] = store[row['raw_data_key']]
store.close()

dates = pd.Series(list(contracts.keys()),dtype=str)


pricing settings:
Actual/365 (Fixed) day counter
New York stock exchange calendar
compounding: continuous
frequency: annual


contracts data available for SPY:
0      2024-10-11
1      2024-10-10
2      2024-10-09
3      2024-10-08
4      2024-10-07
          ...    
352    2023-05-18
353    2023-05-17
354    2023-05-16
355    2023-05-15
356    2023-05-12
Name: 0, Length: 357, dtype: object

                  raw_data_key               surface_key  \
0    /date_2024_10_11/raw_data  /date_2024_10_11/surface   
1    /date_2024_10_10/raw_data  /date_2024_10_10/surface   
2    /date_2024_10_09/raw_data  /date_2024_10_09/surface   
3    /date_2024_10_08/raw_data  /date_2024_10_08/surface   
4    /date_2024_10_07/raw_data  /date_2024_10_07/surface   
..                         ...                       ...   
352  /date_2023_05_18/raw_data                       NaN   
353  /date_2023_05_17/raw_data                       NaN   
354  /date_2023_05_16/raw_data                       NaN   
355 

In [2]:
historical_spots

,spot_price
date,
2024-10-11,579.5800
2024-10-10,576.1300
2024-10-09,577.1400
2024-10-08,573.1700
2024-10-07,567.8000
...,...
1999-11-05,137.8750
1999-11-04,136.5312
1999-11-03,135.5000


In [3]:
date = dates[0]
df = contracts[date]
spot = float(historical_spots.loc[date].iloc[0])
print(spot)

473.88


In [4]:
df = df.copy()
df

,contractID,symbol,expiration,strike,type,last,mark,bid,bid_size,ask,ask_size,volume,open_interest,date,implied_volatility,delta,gamma,theta,vega,rho
0,SPY240109C00405000,SPY,2024-01-09,405.00,call,0.00,68.75,68.63,100,68.86,100,0,0,2024-01-09,1.16646,1.00000,0.00000,-0.21583,0.00000,0.01109
1,SPY240109P00405000,SPY,2024-01-09,405.00,put,0.00,0.01,0.00,0,0.01,4123,0,0,2024-01-09,1.01264,-0.00138,0.00018,-0.20738,0.00112,-0.00002
2,SPY240109C00406000,SPY,2024-01-09,406.00,call,0.00,67.75,67.64,100,67.86,100,0,0,2024-01-09,1.14953,1.00000,0.00000,-0.21637,0.00000,0.01112
3,SPY240109P00406000,SPY,2024-01-09,406.00,put,0.01,0.01,0.00,0,0.01,4123,1,1465,2024-01-09,0.99801,-0.00140,0.00019,-0.20690,0.00114,-0.00002
4,SPY240109C00407000,SPY,2024-01-09,407.00,call,0.00,66.75,66.64,100,66.86,100,0,0,2024-01-09,1.13259,1.00000,0.00000,-0.21690,0.00000,0.01115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7257,SPY261218P00730000,SPY,2026-12-18,730.00,put,0.00,256.00,253.50,202,258.50,202,0,0,2024-01-09,0.46906,-0.47600,0.00104,-0.00134,3.23703,-14.16974
7258,SPY261218C00740000,SPY,2026-12-18,740.00,call,0.88,3.50,2.00,1,5.00,202,0,2,2024-01-09,0.11556,0.08722,0.00169,-0.04549,1.28950,1.11338
7259,SPY261218P00740000,SPY,2026-12-18,740.00,put,0.00,266.00,263.50,202,268.50,202,0,0,2024-01-09,0.47958,-0.47600,0.00102,-0.00178,3.23704,-14.46480
7260,SPY261218C00750000,SPY,2026-12-18,750.00,call,2.02,1.52,1.00,1,2.05,9,5,1504,2024-01-09,0.10047,0.04768,0.00122,-0.02501,0.80709,0.62016


In [5]:
columns_to_convert = ['strike', 'last', 'mark',
       'bid', 'bid_size', 'ask', 'ask_size', 'volume', 'open_interest',
       'implied_volatility', 'delta', 'gamma', 'theta', 'vega', 'rho']
df[columns_to_convert] = df[
    columns_to_convert].apply(pd.to_numeric, errors='coerce')

df['expiration'] = pd.to_datetime(df['expiration'],format='%Y-%m-%d')
df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d')
df['days_to_maturity'] = df['expiration'] - df['date']
df['days_to_maturity'] = df['days_to_maturity'] / np.timedelta64(1, 'D')
df['days_to_maturity'] = df['days_to_maturity'].astype('int64')
df = df[(df['days_to_maturity']>=30)&(df['days_to_maturity']<=400)]

df = df[df['volume']>0].copy()
df['spot_price'] = spot

df['moneyness'] = ms.vmoneyness(df['spot_price'],df['strike'],df['type'])
df = df[(df['moneyness']<0)&(df['moneyness']>-0.5)]
indexed = df.copy().set_index(['strike','days_to_maturity'])

T = np.sort(df['days_to_maturity'].unique()).tolist()
K = np.sort(df['strike'].unique()).tolist()
volume_heatmap = pd.DataFrame(
    np.full((len(K), len(T)), np.nan), index=K, columns=T)
for k in K:
    for t in T:
        try:
            volume_heatmap.loc[k,t] = indexed.loc[(k,t),'volume']
        except Exception:
            pass
        
        
hottest_contracts = pd.DataFrame(
    volume_heatmap.unstack().sort_values(
        ascending=False)).head(50).reset_index()
hottest_contracts.columns = ['t','k','volume']
T = np.sort(hottest_contracts['t'].unique()).tolist()
K = np.sort(hottest_contracts['k'].unique()).tolist()

vol_matrix = pd.DataFrame(
    np.full((len(K),len(T)),np.nan),
    index = K,
    columns = T
)
for k in K:
    for t in T:
        try:
            vol_matrix.loc[k,t] = indexed.loc[(k,float(t)),'implied_volatility']
        except Exception:
            pass

vol_matrix = vol_matrix.dropna().copy()
T = vol_matrix.columns.tolist()
K = vol_matrix.index.tolist()

cols_to_map = [
        'contractID', 'symbol', 'expiration', 'type', 'last', 'mark',
        'bid', 'bid_size', 'ask', 'ask_size', 'volume', 'open_interest', 'date',
        'implied_volatility', 'delta', 'gamma', 'theta', 'vega', 'rho',
        'spot_price', 'moneyness'
]
for col in cols_to_map:
    for i,row in hottest_contracts.iterrows():
        hottest_contracts.at[i,col] = indexed.loc[(row['k'],row['t']),col]
        
hottest_contracts = hottest_contracts.rename(
    columns={'t':'days_to_maturity','k':'strike_price'}).copy()

vol_matrix

,31,38,66,101,143,164,346
415.0,0.22806,0.21800,0.20519,0.20138,0.19971,0.20001,0.21327
420.0,0.21541,0.20733,0.19757,0.19498,0.19437,0.19498,0.21022
425.0,0.20413,0.19742,0.18995,0.18873,0.18888,0.18995,0.20702
430.0,0.19163,0.18599,0.18263,0.18263,0.18355,0.18507,0.20413
440.0,0.16876,0.16739,0.16815,0.17044,0.17318,0.17562,0.19818
450.0,0.14940,0.14986,0.15428,0.15870,0.16343,0.16647,0.19254
455.0,0.14041,0.14178,0.14742,0.15306,0.15870,0.16205,0.18995
460.0,0.13248,0.13370,0.14102,0.14773,0.15428,0.15779,0.18721
465.0,0.12456,0.12654,0.13462,0.14254,0.15032,0.15413,0.18431
470.0,0.11770,0.11983,0.12913,0.13827,0.14651,0.15062,0.18233
